In [ ]:
import sys,os
sys.path.insert(1,'/rpod3/lsiemens/offline_OMEGA_SYGMA/OMEGA_SYGMA_critter')
os.environ["SYGMADIR"] = "/rpod3/lsiemens/offline_OMEGA_SYGMA/OMEGA_SYGMA_critter"
import matplotlib
#matplotlib.use('nbagg')
%matplotlib nbagg
import sygma as s

import modules.widget_framework as framework
from IPython.html import widgets
from IPython.display import display, clear_output
from matplotlib import pyplot
reload (framework)

In [ ]:
frame = framework.framework()
frame.set_default_display_style(padding="0.25em")
frame.set_default_io_style(padding="0.25em", margin="0.25em")

In [ ]:
states = ["run_sim", "plot_totmasses", "plot_mass", "plot_spectro", "plot_mass_range"]

frame.add_state(states)

In [ ]:
frame.add_display_object("window")
frame.add_io_object("title")
frame.add_display_object("widget")

###Sim page###
frame.add_display_object("sim_page")

frame.add_display_object("mass_Z_group")
frame.add_io_object("init_Z")
frame.add_io_object("mass_gas")

frame.add_display_object("time_group")
frame.add_io_object("t_end")
frame.add_io_object("dt")

frame.add_display_object("imf_type_group")
frame.add_io_object("imf_type")
frame.add_io_object("imf_alpha")

frame.add_display_object("imf_mass_group")
frame.add_io_object("imf_mass_min")
frame.add_io_object("imf_mass_max")

frame.add_display_object("sn1a_group")
frame.add_io_object("use_sn1a")
frame.add_io_object("sn1a_rates")
frame.add_io_object("run_sim")

frame.add_io_object("plot_type")

frame.set_state_children("window", ["title", "widget"])
frame.set_state_children("widget", ["sim_page"], titles=["Simulation"])
frame.set_state_children("sim_page", ["mass_Z_group", "time_group", "imf_type_group", "imf_mass_group", "sn1a_group", "run_sim", "plot_type"])
frame.set_state_children("mass_Z_group", ["mass_gas", "init_Z"])
frame.set_state_children("time_group", ["t_end", "dt"])
frame.set_state_children("imf_type_group", ["imf_type", "imf_alpha"])
frame.set_state_children("imf_mass_group", ["imf_mass_min", "imf_mass_max"])
frame.set_state_children("sn1a_group", ["use_sn1a", "sn1a_rates"])

In [ ]:
frame.set_state_data("sygma", None)

frame.set_state_attribute('window', visible=True)
frame.set_state_attribute('title', visible=True, value="<h1>SYGMA<h1>")
frame.set_state_attribute('widget', visible=True)

frame.set_state_attribute('sim_page', visible=True)
frame.set_state_attribute("mass_Z_group", visible=True)
frame.set_state_attribute("mass_gas", visible=True, description="Gas cloud mass:", value="1.0")
frame.set_state_attribute('init_Z', visible=True, description="Set inital Z: ", options=["0.0", "0.0001", "0.006", "0.01", "0.02"], selected_label="0.02")

frame.set_state_attribute('time_group', visible=True)
frame.set_state_attribute('t_end', visible=True, description="Set duration: ", value="1.0e10")
frame.set_state_attribute('dt', visible=True, description="Set time step: ", value="1.0e7")

frame.set_state_attribute('imf_type_group', visible=True)
frame.set_state_attribute('imf_type', visible=True, description="IMF type: ", options=['salpeter', 'chabrier', 'kroupa', 'alphaimf'])
frame.set_state_attribute('imf_alpha', description="Set alpha: ", value=2.35, min=0, max=5)

frame.set_state_attribute("imf_mass_group", visible=True)
frame.set_state_attribute('imf_mass_min', visible=True, description="IMF mass min: ", value="1.0")
frame.set_state_attribute('imf_mass_max', visible=True, description="IMF mass max: ", value="30.0")

frame.set_state_attribute('sn1a_group', visible=True)
frame.set_state_attribute('use_sn1a', visible=True, description="Include SNe Ia: ")
frame.set_state_links("sn1a_link", [("use_sn1a", "value"), ("sn1a_rates", "visible")], directional=True)

frame.set_state_attribute('sn1a_rates', description="SNe Ia rates: ", options=['maoz', 'wiersmaexp', 'wiersmagauss'])

frame.set_state_attribute('run_sim', visible=True, description="Run simulation")

frame.set_state_attribute('plot_type', visible=True, disabled=True, description="Plot type: ", options=["Total mass", "Species mass", "Species spectroscopic", "Mass range contributions"])
frame.set_state_attribute("plot_type", states, disabled=False)


def sel_imf_type(attribute, value):
    if value=="alphaimf":
        frame.set_attributes("imf_alpha", visible=True)
    else:
        frame.set_attributes("imf_alpha", visible=False)

def run_simulation(widget):
    mgal = float(frame.get_attribute("mass_gas", "value"))
    iniZ = float(frame.get_attribute("init_Z", "value"))
    imf_type = frame.get_attribute("imf_type", "value")
    alphaimf = frame.get_attribute("imf_alpha", "value")
    mass_min = float(frame.get_attribute("imf_mass_min", "value"))
    mass_max = float(frame.get_attribute("imf_mass_max", "value"))
    imf_bdys = [mass_min, mass_max]
    sn1a_on = frame.get_attribute("use_sn1a", "value")
    sn1a_rate = frame.get_attribute("sn1a_rates", "value")
    dt = float(frame.get_attribute("dt", "value"))
    tend = float(frame.get_attribute("t_end", "value"))
    data=s.sygma(mgal=mgal, iniZ=iniZ, imf_type=imf_type, alphaimf=alphaimf, imf_bdys=imf_bdys, sn1a_on=sn1a_on,
                 sn1a_rate=sn1a_rate, dt=dt,tend=tend)
    frame.set_state_data("sygma", data)
    frame.set_state("run_sim")
    
def sel_plot_type(attribute, value):
    if value=="Total mass":
        frame.set_state("plot_totmasses")
    elif value=="Species mass":
        frame.set_state("plot_mass")
    elif value=="Species spectroscopic":
        frame.set_state("plot_spectro")
    elif value=="Mass range contributions":
        frame.set_state("plot_mass_range")
        
frame.set_state_callbacks("imf_type", sel_imf_type)
frame.set_state_callbacks("run_sim", run_simulation, attribute=None, type="on_click")
frame.set_state_callbacks("plot_type", sel_plot_type)

frame.set_object("window", widgets.Box())
frame.set_object("title", widgets.HTML())
frame.set_object("widget", widgets.Tab())

frame.set_object("sim_page", widgets.VBox())
frame.set_object("mass_Z_group", widgets.HBox())
frame.set_object("mass_gas", widgets.Text())
frame.set_object("init_Z", widgets.Dropdown(options=["0.02"]))# option 0.02 is included since selection_label is set and will be called before options in set_state_attributes

frame.set_object("time_group", widgets.HBox())
frame.set_object("t_end", widgets.Text())
frame.set_object("dt", widgets.Text())

frame.set_object("imf_type_group", widgets.HBox())
frame.set_object("imf_type", widgets.Dropdown())
frame.set_object("imf_alpha", widgets.FloatSlider())

frame.set_object("imf_mass_group", widgets.HBox())
frame.set_object("imf_mass_min", widgets.Text())
frame.set_object("imf_mass_max", widgets.Text())

frame.set_object("sn1a_group", widgets.HBox())
frame.set_object("use_sn1a", widgets.Checkbox())
frame.set_object("sn1a_rates", widgets.Dropdown())

frame.set_object("run_sim", widgets.Button())

frame.set_object("plot_type", widgets.Dropdown())


In [ ]:
frame.display_object("window")